# Data Correction [stock splits]

While exploring the data, I've noticed that prices were not adjusted for stock split events. The stock splits will affect calculation of any ratios based on prices.

In this notebook, I will adjust the prices for stock splits and save the the data to a new file endig with 'v1'

In [1]:
# Importing the necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
# Load the data

filepath_finance_data = '../Data/Datasets/updated_finance_data.csv'
finance_data = pd.read_csv(filepath_finance_data)

filepath_split_data = '../Data/Datasets/split_history.csv'

split_data = pd.read_csv(filepath_split_data)


In [3]:
# Checking the data
# ordering all columns in alphabetical order

finance_data = finance_data.reindex(sorted(finance_data.columns), axis=1)

# for easier viewing, I will make sure all float values are rounded to 2 decimal places and have commas

pd.options.display.float_format = '{:,.2f}'.format

# sorting the finance data by symbol (ascending) and date_x (ascending)

finance_data = finance_data.sort_values(by=['symbol', 'date_x'], ascending=[True, True])

# viewing the first 10 rows of the data

finance_data.head(10)

,addTotalDebt,assetGrowth,assetTurnover,averageInventory,averagePayables,averageReceivables,beta,bookValuePerShare,bookValueperShareGrowth,calendarYear,...,threeYNetIncomeGrowthPerShare,threeYOperatingCFGrowthPerShare,threeYRevenueGrowthPerShare,threeYShareholdersEquityGrowthPerShare,totalDebtToCapitalization,updatedFromDate,weightedAverageSharesDilutedGrowth,weightedAverageSharesGrowth,workingCapital,zip
39,36403000000,0.13,0.28,"2,197,000,000.00","34,098,500,000.00","17,084,500,000.00",1.24,5.28,0.12,2015,...,0.54,1.14,0.80,0.53,0.23,2024-11-25,-0.02,-0.02,"9,792,000,000.00",95014
38,43871000000,-0.00,0.22,"2,339,500,000.00","30,580,000,000.00","13,807,000,000.00",1.24,5.57,0.05,2015,...,0.32,0.54,0.67,0.42,0.25,2024-11-25,-0.01,-0.01,"9,162,000,000.00",95014
37,54418000000,0.05,0.18,"2,219,000,000.00","24,816,500,000.00","10,637,500,000.00",1.24,5.48,-0.01,2015,...,0.38,0.68,0.62,0.29,0.30,2024-11-25,-0.01,-0.01,"5,668,000,000.00",95014
36,64328000000,0.06,0.18,"2,195,500,000.00","30,982,000,000.00","13,609,500,000.00",1.24,5.28,-0.04,2015,...,0.57,0.72,0.67,0.17,0.35,2024-11-25,-0.02,-0.01,"8,768,000,000.00",95014
35,62963000000,0.01,0.26,"2,400,000,000.00","34,401,000,000.00","14,901,000,000.00",1.24,5.77,0.09,2016,...,0.66,0.39,0.65,0.19,0.33,2024-11-25,-0.02,-0.02,"127,000,000.00",95014
34,79872000000,0.04,0.17,"2,366,000,000.00","29,205,000,000.00","12,591,000,000.00",1.24,5.91,0.03,2016,...,0.31,0.11,0.38,0.15,0.38,2024-11-25,-0.01,-0.01,"19,327,000,000.00",95014
33,84935000000,0.00,0.14,"2,056,000,000.00","25,708,000,000.00","11,971,500,000.00",1.24,5.81,-0.02,2016,...,0.33,0.60,0.42,0.21,0.40,2024-11-25,-0.01,-0.01,"22,275,000,000.00",95014
32,87032000000,0.05,0.15,"1,981,500,000.00","31,806,000,000.00","20,506,500,000.00",1.24,5.97,0.03,2016,...,0.42,0.92,0.47,0.22,0.40,2024-11-25,-0.01,-0.01,"27,863,000,000.00",95014
31,87549000000,0.03,0.24,"2,422,000,000.00","37,902,000,000.00","28,638,000,000.00",1.24,6.25,0.05,2017,...,0.62,0.42,0.61,0.21,0.40,2024-11-25,-0.01,-0.01,"19,202,000,000.00",95014
30,98522000000,0.01,0.16,"2,811,000,000.00","33,541,500,000.00","24,294,500,000.00",1.24,6.41,0.03,2017,...,0.26,0.09,0.36,0.31,0.42,2024-11-25,-0.01,-0.01,"28,648,000,000.00",95014


In [4]:
# removing any rows from stock split data where the date is less than 2014-01-01

split_data = split_data[split_data['date'] >= '2013-01-01']

split_data

,date,label,numerator,denominator,symbol
0,2020-08-31,"August 31, 20",4.00,1.00,AAPL
1,2014-06-09,"June 09, 14",7.00,1.00,AAPL
5,2013-01-02,"January 02, 13","5,000.00","2,399.00",ABT
37,2014-10-01,"October 01, 14","1,139.00","1,000.00",ADP
50,2018-03-19,"March 19, 18",2.00,1.00,AFL
...,...,...,...,...,...
911,2013-12-02,"December 02, 13",313.00,250.00,TT
916,2021-06-17,"June 17, 21",10.00,1.00,TTD
937,2014-06-09,"June 09, 14",2.00,1.00,UNP
943,2015-03-19,"March 19, 15",4.00,1.00,V


In [5]:
# Create a new column called 'adjustment_factor' which is the ratio calculated as 'denominator' / 'numerator'
split_data.loc[:, 'adjustment_factor'] = split_data['denominator'] / split_data['numerator']

# drop all columns except 'symbol', 'date', 'adjustment_factor'

split_data = split_data[['symbol', 'date', 'adjustment_factor']]

split_data

,symbol,date,adjustment_factor
0,AAPL,2020-08-31,0.25
1,AAPL,2014-06-09,0.14
5,ABT,2013-01-02,0.48
37,ADP,2014-10-01,0.88
50,AFL,2018-03-19,0.50
...,...,...,...
911,TT,2013-12-02,0.80
916,TTD,2021-06-17,0.10
937,UNP,2014-06-09,0.50
943,V,2015-03-19,0.25
